# 循环神经网络的从零开始实现

In [1]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [2]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)
# 复习
# train_iter是一个迭代器 每次调用返回一对(X, Y)
# X 是输入序列的批次，形状为 (batch_size, num_steps)。
# Y 是目标序列的批次，形状为 (batch_size, num_steps)。
# vocab 是一个词汇表对象，包含了数据集中所有唯一词元及其对应的索引。
# 包含以下属性和方法：
# vocab[token]：返回词元 token 对应的索引。
# vocab.to_tokens(index)：将索引转换回词元。
# len(vocab)：返回词汇表中词元的数量。

In [3]:
# 查看词汇表中的一些词元
# 默认是字符集映射的
temp_train_iter, temp_vocab = d2l.load_data_time_machine(batch_size, num_steps)
tx,ty = next(iter(temp_train_iter))
print(tx)
print(tx.shape)
print(ty)
print(temp_vocab.idx_to_token[:10])  # 显示前10个词元
print(temp_vocab.token_to_idx)  # 显示词元到索引的映射

tensor([[13,  4, 15,  ...,  4, 22,  2],
        [ 2,  1, 21,  ...,  1,  5,  6],
        [ 4,  6,  1,  ..., 12, 19,  4],
        ...,
        [10,  2, 10,  ..., 16,  7, 10],
        [12,  5,  6,  ...,  1,  2, 24],
        [ 5,  8, 13,  ...,  4,  3,  1]])
torch.Size([32, 35])
tensor([[ 4, 15,  9,  ..., 22,  2, 12],
        [ 1, 21, 14,  ...,  5,  6,  1],
        [ 6,  1, 16,  ..., 19,  4, 11],
        ...,
        [ 2, 10,  1,  ...,  7, 10,  2],
        [ 5,  6,  2,  ...,  2, 24, 20],
        [ 8, 13,  3,  ...,  3,  1, 11]])
['<unk>', ' ', 'e', 't', 'a', 'i', 'n', 'o', 's', 'h']
{'<unk>': 0, ' ': 1, 'e': 2, 't': 3, 'a': 4, 'i': 5, 'n': 6, 'o': 7, 's': 8, 'h': 9, 'r': 10, 'd': 11, 'l': 12, 'm': 13, 'u': 14, 'c': 15, 'f': 16, 'w': 17, 'g': 18, 'y': 19, 'p': 20, 'b': 21, 'v': 22, 'k': 23, 'x': 24, 'z': 25, 'j': 26, 'q': 27}


In [4]:
# 独热编码
# 将 [0, 2] 转换为独热编码，使用词汇表 vocab 的长度作为独热编码的深度。
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [5]:
# 小批量数据形状是（批量大小，时间步数）
# one_hot函数将其转化为三维张量  （时间步数，批量大小，词表大小）
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape

torch.Size([5, 2, 28])

In [6]:
# 初始化模型参数
def get_params(vocab_size, num_hiddens, device):
    # 输出输出等于词表大小
    num_inputs = num_outputs = vocab_size

    # 定义一个函数，用于生成正态分布随机数并乘以0.01
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [7]:
# 在初始化时返回隐状态
# 形状（批量大小，隐藏单元数）
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [8]:
# 在一个时间步内计算隐状态和输出
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小) 进行独热编码的结果
    # 参数解包
    W_xh, W_hh, b_h, W_hq, b_q = params
    # 隐状态解包
    H, = state
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)
# torch.cat(outputs, dim=0): 将所有时间步的输出在维度0上连接，得到形状为(时间步数量*批量大小，词表大小)的张量
# (H,): 返回最终的隐藏状态作为元组

In [9]:
# 循环神经网络类
class RNNModelScratch: #@save
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        # 即上方的函数
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [10]:
# 检查输入输出是否符合对应的维度
# len(vocab)=28
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
# X是上面定义的测试 (2,5)
# state是包含1个元素的元组，元素形状为 (2, 512)

state = net.begin_state(X.shape[0], d2l.try_gpu())
# 在 __call__ 方法中，X.T 形状变为 (5, 2)
# 独热编码后形状为 (5,2,28)
# 在 rnn 函数中，按时间步迭代:
# 每个时间步的 X 形状是 (2, 28)
# 隐藏状态 H 形状是 (2, 512)
# 每个时间步的输出 Y 形状是 (2, 28)
# 所有时间步的输出连接后 torch.cat(outputs, dim=0) 形状为 (5*2, 28) = (10, 28)
# 最终隐藏状态 new_state 仍是包含1个元素的元组，元素形状为 (2, 512)
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

# RNN模型中的形状详解

## 基本参数和变量

### 词汇和隐藏状态维度
- **vocab_size**: 词汇表大小
- **num_hiddens**: 隐藏单元数量

### 输入数据
- **X**: 形状为 `(batch_size, num_steps)`
  - 第一维：批量大小，表示同时处理的序列数量
  - 第二维：时间步数，表示每个序列的长度

## 模型参数形状

### 权重矩阵和偏置
- **W_xh**: `(vocab_size, num_hiddens)`
  - 输入到隐藏层的权重
- **W_hh**: `(num_hiddens, num_hiddens)`
  - 隐藏层到隐藏层的权重（循环连接）
- **b_h**: `(num_hiddens,)`
  - 隐藏层的偏置
- **W_hq**: `(num_hiddens, vocab_size)`
  - 隐藏层到输出层的权重
- **b_q**: `(vocab_size,)`
  - 输出层的偏置

## 数据流转换过程

### 初始化
- **state**: 返回元组 `(H,)`，其中 H 形状为 `(batch_size, num_hiddens)`
  - 每个批次样本都有自己的隐藏状态向量

### 前向传播过程
1. **输入转换**:
   - **X.T**: `(num_steps, batch_size)`
   - **F.one_hot(X.T, vocab_size)**: `(num_steps, batch_size, vocab_size)`

2. **RNN循环处理**:
   - 迭代每个时间步:
     - 单个时间步的 **X**: `(batch_size, vocab_size)`
     - 隐藏状态 **H**: `(batch_size, num_hiddens)`
     - 单个时间步的输出 **Y**: `(batch_size, vocab_size)`

3. **最终输出**:
   - **Y** (连接所有时间步): `(num_steps * batch_size, vocab_size)`
     - 注意这里的维度是将所有时间步的输出堆叠在一起
   - **new_state**: 元组 `(H,)`，其中 H 形状为 `(batch_size, num_hiddens)`
     - 只保留最后一个时间步的隐藏状态

## 维度变换的关键点
1. 转置操作 `X.T` 将时间步放在第一维，便于按时间步迭代
2. 独热编码将整数索引转换为向量表示
3. 输出连接 (`torch.cat`) 将所有时间步结果连接成一个大张量
4. 隐藏状态保持为元组，便于扩展到更复杂的RNN变种(如LSTM)

In [19]:
# 进行预测 参数内容(前缀字符串 要生成的字符数量 网络模型 词汇表 计算设备)
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    # batch_size=1 表示一次只预测一个序列
    # 返回形状为 (1, num_hiddens) 的隐藏状态张量，作为元组 (H,)
    state = net.begin_state(batch_size=1, device=device)
    # outputs 是一个列表，包含前缀第一个字符的索引 t对应的索引是3 所以 outputs = [3]
    outputs = [vocab[prefix[0]]]
    # 获得outputs列表中的最后一个元素 并将其转化成(1,1)的向量
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    # 虽然outputs初始化时只有一个元素，但在每次生成新的字符后，这个列表会不断增长。
    # 因此取outputs列表中的最后一个元素是为了获取最近生成的字符索引，并将其作为下一步的输入。
    
    # 遍历 prefix 中从第二个字符开始的每个字符。
    # 进行预热 
    # 预热是指在正式开始生成新字符之前，使用前缀字符来驱动 RNN 网络。
    # 通过预热期，RNN 网络可以根据前缀字符来更新其隐藏状态，从而提供一个更好的初始条件给后续的字符生成过程。
    # 这样可以使生成的新字符更符合前缀的上下文。
    for y in prefix[1:]: 
        _, state = net(get_input(), state)
        # 更新隐藏状态 state 并将当前字符的索引添加到 outputs 列表。
        outputs.append(vocab[y])
    # 预测num_preds步    
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    # 展示内容
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [20]:
# 测试输出
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())

'time traveller vwzs<unk>f<unk>dlv'

In [21]:
# 梯度裁剪函数
def grad_clipping(net, theta):  
    # 获取所有参数
    # nn.Module实例通过遍历获取其中的参数列表
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        # 如果不是的话就假设有这个参数
        params = net.params
    # 计算所有梯度参数的L2范数
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        # 如果梯度范数 norm 超过了阈值 theta，则对每个参数的梯度进行缩放(乘theta / norm)。
        for param in params:
            param.grad[:] *= theta / norm

In [ ]:
# 训练代码从零开始实现
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # 对于 nn.GRU，隐状态是一个单独的张量。我们需要在每个时间步之后调用 detach_() 方法，将隐状态从计算图中分离出来。
                state.detach_()
            else:
                #对于 nn.LSTM，隐状态是一个包含两个张量的元组，分别表示短期记忆（cell state）和长期记忆（hidden state）。
                # 我们需要对元组中的每个张量调用 detach_() 方法，将它们从计算图中分离出来。
                for s in state:
                    s.detach_()

        # 将目标张量 Y 转置并展平为一维张量，以便用于计算损失
        # 在时间序列模型中，输出通常是一个二维张量，其中一维是时间步，另一维是批量大小
        y = Y.T.reshape(-1)
        # 将输入和目标数据移动到指定设备上
        X, y = X.to(device), y.to(device)
        # 计算模型的输出和新的隐状态
        y_hat, state = net(X, state)
        # 计算损失
        l = loss(y_hat, y.long()).mean()
        # 如果 updater 是一个 PyTorch 优化器，则清除梯度，反向传播计算梯度，进行梯度裁剪，并更新模型参数。
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()


In [ ]:
# 使用高级API实现训练器
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))

In [ ]:
# 进行训练
num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

In [ ]:
# 使用随机抽样方法
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu(),
          use_random_iter=True)